<h3>Table prep</h3>

In [1]:
import pandas as pd
import sqlite3
import numpy as np

db = sqlite3.connect('E:/GitHub Repos/TensorFlow training/SoccerDataset/database.sqlite')
df_train = {}

In [2]:
player_df = pd.read_sql_query('SELECT * FROM player', db)
df_train['player'] = player_df.sample(n=100)
df_train['player'].head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
6359,6367,11801,Luis Olim,20543,1981-08-27 00:00:00,177.80,152
4587,4591,28901,James McArthur,171972,1987-10-07 00:00:00,177.80,141
8726,8740,19232,Rafael Miranda,165586,1984-08-11 00:00:00,177.80,165
11001,11017,473857,Zana Allee,192769,1994-03-01 00:00:00,165.10,139
2900,2904,46359,Eggert Jonsson,172398,1988-08-18 00:00:00,187.96,159


In [3]:
player_attr_df = pd.read_sql_query('SELECT * FROM player_attributes', db)

In [4]:
boolean_series = player_attr_df.player_api_id.isin(df_train['player'].player_api_id)
df_train['player_attributes'] = player_attr_df[boolean_series]
df_train['player_attributes'].head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
5023,5024,186961,109121,2016-03-10 00:00:00,77.0,79.0,right,medium,medium,52.0,...,59.0,79.0,12.0,16.0,16.0,15.0,15.0,10.0,13.0,5.0
5024,5025,186961,109121,2016-02-04 00:00:00,77.0,79.0,right,medium,medium,52.0,...,59.0,79.0,12.0,16.0,16.0,15.0,15.0,10.0,13.0,5.0
5025,5026,186961,109121,2016-01-21 00:00:00,77.0,79.0,right,medium,medium,52.0,...,59.0,79.0,12.0,16.0,16.0,15.0,15.0,10.0,13.0,5.0
5026,5027,186961,109121,2016-01-07 00:00:00,77.0,80.0,right,medium,medium,52.0,...,59.0,79.0,12.0,16.0,16.0,15.0,15.0,10.0,13.0,5.0
5027,5028,186961,109121,2015-10-02 00:00:00,77.0,81.0,right,medium,medium,52.0,...,59.0,79.0,12.0,16.0,16.0,15.0,15.0,10.0,13.0,5.0


In [5]:
for k, v in df_train.items():
    df_train[k] = df_train[k].replace([np.inf, -np.inf], np.nan)
    df_train[k] = df_train[k].dropna()

In [6]:
df_train['player'][df_train['player'].isna().any(axis=1)]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight


In [7]:
df_train['player_attributes'][df_train['player_attributes'].isna().any(axis=1)]

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes


<h3>Starting SDV</h3>

In [8]:
from sdv import Metadata

metadata = Metadata()
metadata

Metadata
  root_path: .
  tables: []
  relationships:

In [9]:
metadata.add_table(
    name='player',
    data=df_train['player'],
    primary_key='player_api_id'
)

In [10]:
metadata.add_table(
    name='player_attributes',
    data=df_train['player_attributes'],
    primary_key='id',
    parent='player',
    foreign_key='player_api_id'
)

In [11]:
metadata.to_dict()

{'tables': {'player': {'fields': {'id': {'type': 'numerical',
     'subtype': 'integer'},
    'player_api_id': {'type': 'id', 'subtype': 'integer'},
    'player_name': {'type': 'categorical'},
    'player_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'birthday': {'type': 'categorical'},
    'height': {'type': 'numerical', 'subtype': 'float'},
    'weight': {'type': 'numerical', 'subtype': 'integer'}},
   'primary_key': 'player_api_id'},
  'player_attributes': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'player_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'player_api_id': {'type': 'id',
     'subtype': 'integer',
     'ref': {'table': 'player', 'field': 'player_api_id'}},
    'date': {'type': 'categorical'},
    'overall_rating': {'type': 'numerical', 'subtype': 'float'},
    'potential': {'type': 'numerical', 'subtype': 'float'},
    'preferred_foot': {'type': 'categorical'},
    'attacking_work_rate': {'type': 'categorical'},
    'def

In [147]:
#metadata = {'tables': {'player': {'fields': {'id': {'type': 'numerical', 'subtype': 'integer'}, 'player_api_id': {'type': 'id', 'subtype': 'integer'}, 'player_name': {'type': 'categorical'}, 'player_fifa_api_id': {'type': 'id', 'subtype': 'integer'}, 'birthday': {'type': 'categorical'}, 'height': {'type': 'numerical', 'subtype': 'float'}, 'weight': {'type': 'numerical', 'subtype': 'integer'}}, 'primary_key': 'player_api_id'}, 'player_attributes': {'fields': {'id': {'type': 'id', 'subtype': 'integer'}, 'player_fifa_api_id': {'type': 'id', 'subtype': 'integer', 'ref': {'table': 'player', 'field': 'player_fifa_api_id'}}, 'player_api_id': {'type': 'id', 'subtype': 'integer', 'ref': {'table': 'player', 'field': 'player_api_id'}}, 'overall_rating': {'type': 'numerical', 'subtype': 'float'}, 'potential': {'type': 'numerical', 'subtype': 'float'}, 'preferred_foot': {'type': 'categorical'}}, 'primary_key': 'id'}}}

In [148]:
#df_train['player_attributes'] = df_train['player_attributes'][['id', 'player_fifa_api_id', 'player_api_id', 'overall_rating', 'potential', 'preferred_foot']]

In [12]:
from sdv.relational import HMA1

model = HMA1(metadata)
model.fit(df_train)

In [13]:
new_data = model.sample()

In [14]:
new_data

{'player':       id  player_api_id            player_name  player_fifa_api_id  \
 0   8997              0          Ricardo Nunes              117032   
 1   5296              1       Jonathan Correia              127714   
 2   9248              2     Saidi Ntibazonkiza              196939   
 3   7622              3  Nicolas Maurice-Belay              187301   
 4   2633              4      Dominik Reinhardt              143363   
 ..   ...            ...                    ...                 ...   
 95  4373             95         James McArthur              105557   
 96   467             96       Andrew Barrowman              187511   
 97  3397             97          Frazer Wright              237497   
 98  6711             98         Leonard Kweuke              226305   
 99  7583             99           Ogenyi Onazi              122305   
 
                birthday      height  weight  
 0   1990-01-04 00:00:00  187.684687     162  
 1   1987-02-26 00:00:00  181.188543     1

In [16]:
def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

def get_table_order(db):
    '''This method returns the order of tables to be generated. Tables have relations between each other (FK) that needs to be created first in order to generate consistend data. 
    TODO: Remove pk; fk stuff from here. Method already big enough, will get own method to call when needed. '''
    rows = db.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
    tables = [row[0].lower() for row in rows]
    
    if "sqlite_sequence" in tables:
        tables.remove('sqlite_sequence')
    
    fk_relation = {}
    pk_relation = {}
    
    table_order = []
    
    for table in tables:
        rows = db.execute("PRAGMA table_info({})".format(sql_identifier(table)))
        attributes = rows.fetchall()
        
        pk = []
        for attr in attributes:
            if attr[5] == 1:
                pk.append(attr[1])    
        pk_relation[table] = pk
        
        rows = db.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
        foreign_key_list = rows.fetchall()
        fkeys = []
        for fk in foreign_key_list:
            fkeys.append({'table': fk[2].lower(), 'origin': fk[3], 'dest': fk[4]})
        fk_relation[table] = fkeys
        
        if not fkeys:
            table_order.append(table)
        else:
            contained = True
            depends = []
            for fk in fkeys:
                if fk['table'] not in depends:
                    depends.append(fk['table'])
                if fk['table'] not in table_order:
                    contained = False
            if contained:
                #table_order.append((table, depends))
                table_order.append(table)
                
                
    maxLoop = 5
    while len(table_order) <= len(tables):
        for table, fkeys in fk_relation.items():
            if table in table_order:
                continue
                
            contained = True
            depends = []
            for fk in fkeys:
                if fk['table'] not in depends:
                    depends.append(fk['table'])
                if fk['table'] not in table_order:
                    contained = False
            if contained:
                table_order.append(table)  
                
        maxLoop -= 1
        if maxLoop <= 0:
            break
        
    return (table_order, pk_relation, fk_relation)


In [32]:
con = sqlite3.connect('E:\GitHub Repos\TensorFlow training\SoccerDataset\database.sqlite')
table_order, pk_relation, fk_relation = get_table_order(con)

In [33]:
pk_relation

{'player_attributes': ['id'],
 'player': ['id'],
 'match': ['id'],
 'league': ['id'],
 'country': ['id'],
 'team': ['id'],
 'team_attributes': ['id']}

In [34]:
fk_relation

{'player_attributes': [{'table': 'player',
   'origin': 'player_api_id',
   'dest': 'player_api_id'},
  {'table': 'player',
   'origin': 'player_fifa_api_id',
   'dest': 'player_fifa_api_id'}],
 'player': [],
 'match': [{'table': 'player',
   'origin': 'away_player_11',
   'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_10', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_9', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_8', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_7', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_6', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_5', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_4', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_3', 'dest': 'player_api_id'},
  {'table': 'player', 'origin': 'away_player_2', 'dest': 'player_api_id'},
  {'table': 'player', 'o

In [39]:
def get_Metadata(con, tables, pk_relation, fk_relation):
    metadata = Metadata()

    for table in tables:
        df = pd.read_sql_query('SELECT * FROM ' + table + ' LIMIT 5', con)

        metadata.add_table(
            name=table,
            data=df,
            primary_key=pk_relation[table][0] #TODO: Könnte sein, dass es mehr prim keys gibt. Dann schauen.
        )

    return metadata

In [40]:
metadata = get_Metadata(con, table_order, pk_relation, fk_relation)

In [41]:
metadata.to_dict()

{'tables': {'player': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'player_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'player_name': {'type': 'categorical'},
    'player_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'birthday': {'type': 'categorical'},
    'height': {'type': 'numerical', 'subtype': 'float'},
    'weight': {'type': 'numerical', 'subtype': 'integer'}},
   'primary_key': 'id'},
  'country': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'name': {'type': 'categorical'}},
   'primary_key': 'id'},
  'team': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'team_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'team_fifa_api_id': {'type': 'numerical', 'subtype': 'integer'},
    'team_long_name': {'type': 'categorical'},
    'team_short_name': {'type': 'categorical'}},
   'primary_key': 'id'},
  'team_attributes': {'fields': {'id': {'type': 'id', 'subtype': 'integer'},
    'team_fifa_api_id': {'typ

In [25]:
df = pd.read_sql_query('SELECT * FROM ' + table_order[0], con)

In [43]:
df.columns

Index(['Id', 'EmployeeName', 'JobTitle', 'BasePay', 'OvertimePay', 'OtherPay',
       'Benefits', 'TotalPay', 'TotalPayBenefits', 'Year', 'Notes', 'Agency',
       'Status'],
      dtype='object')

In [11]:
player_df

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [17]:
metadata.add_table(
    name='player',
    data=player_df,
    primary_key='id'
)

In [22]:
metadata

Metadata
  root_path: .
  tables: ['player']
  relationships:

In [24]:
player_df

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [19]:
from sdv.tabular import GaussianCopula

In [89]:
model = GaussianCopula(primary_key=['id', 'player_fifa_api_id', 'player_api_id'],
                       anonymize_fields= {'player_name': 'name'},
                       field_transformers= {
                           'birthday': 'label_encoding',
                           'height': 'float',
                           'weight': 'integer'
                       })

In [90]:
model.fit(player_df[:100])

E:\ProgramData\Anaconda3\envs\master\lib\site-packages\scipy\stats\_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
E:\ProgramData\Anaconda3\envs\master\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
E:\ProgramData\Anaconda3\envs\master\lib\site-packages\scipy\stats\_continuous_distns.py:4965: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
E:\ProgramData\Anaconda3\envs\master\lib\site-packages\scipy\stats\_distn_infrastructure.py:2429: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu


In [60]:
res = model.sample(10000)

In [61]:
res.describe()

,id,player_api_id,player_fifa_api_id,height,weight
count,10000.00000,10000.00000,10000.00000,10000.000000,10000.000000
mean,4999.50000,4999.50000,4999.50000,181.640037,166.558800
std,2886.89568,2886.89568,2886.89568,6.197387,13.475381
min,0.00000,0.00000,0.00000,153.666390,131.000000
25%,2499.75000,2499.75000,2499.75000,177.441458,157.000000
50%,4999.50000,4999.50000,4999.50000,181.604012,165.000000
75%,7499.25000,7499.25000,7499.25000,185.771068,175.000000
max,9999.00000,9999.00000,9999.00000,204.602744,243.000000


In [62]:
player_df.describe()

,id,player_api_id,player_fifa_api_id,height,weight
count,11060.000000,11060.000000,11060.000000,11060.000000,11060.000000
mean,5537.511392,156582.427215,165664.910488,181.867445,168.380289
std,3197.692647,160713.700624,58649.928360,6.369201,14.990217
min,1.000000,2625.000000,2.000000,157.480000,117.000000
25%,2767.750000,35555.500000,151889.500000,177.800000,159.000000
50%,5536.500000,96619.500000,184671.000000,182.880000,168.000000
75%,8306.250000,212470.500000,203883.250000,185.420000,179.000000
max,11075.000000,750584.000000,234141.000000,208.280000,243.000000


In [63]:
res.player_fifa_api_id.value_counts().max()

1

In [92]:
model.get_metadata().to_dict()

{'fields': {'id': {'type': 'id',
   'subtype': 'integer',
   'transformer': 'integer'},
  'player_api_id': {'type': 'id',
   'subtype': 'integer',
   'transformer': 'integer'},
  'player_name': {'type': 'categorical',
   'transformer': 'one_hot_encoding',
   'pii': True,
   'pii_category': 'name'},
  'player_fifa_api_id': {'type': 'id',
   'subtype': 'integer',
   'transformer': 'integer'},
  'birthday': {'type': 'categorical', 'transformer': 'label_encoding'},
  'height': {'type': 'numerical', 'subtype': 'float', 'transformer': 'float'},
  'weight': {'type': 'numerical',
   'subtype': 'integer',
   'transformer': 'integer'}},
 'constraints': [],
 'model_kwargs': {'GaussianCopula': {'field_distributions': {'player_name#0': 'copulas.univariate.gaussian.GaussianUnivariate',
    'player_name#1': 'copulas.univariate.gaussian.GaussianUnivariate',
    'player_name#2': 'copulas.univariate.gaussian.GaussianUnivariate',
    'player_name#3': 'copulas.univariate.gaussian.GaussianUnivariate',
    

In [2]:
db = sqlite3.connect('E:/GitHub Repos/TensorFlow training/SoccerDataset/salaries.sqlite')
salaries = pd.read_sql_query('SELECT * FROM salaries', db)

In [3]:
salaries

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411,0,400184,,567595.43,567595.43,2011,,San Francisco,
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966,245132,137811,,538909.28,538909.28,2011,,San Francisco,
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739,106088,16452.6,,335279.91,335279.91,2011,,San Francisco,
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916,56120.7,198307,,332343.61,332343.61,2011,,San Francisco,
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134402,9737,182235,,326373.19,326373.19,2011,,San Francisco,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
148649,148650,Roy I Tillery,Custodian,0,0,0,0,0.00,0.00,2014,,San Francisco,PT
148650,148651,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,
148651,148652,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,
148652,148653,Not provided,Not provided,Not Provided,Not Provided,Not Provided,Not Provided,0.00,0.00,2014,,San Francisco,
